In [166]:
rest = 'https://ftx.com'
ws = 'wss://ftx.com/ws/'
key = 'mbv1hpU6ZC634XVx9KOT_I-7CShH1GIgNGkMOG4U'
secret = '3jUCgi35VxpRhN2iKyQsqUjSkL96Agi3VSiguktM'

In [216]:
import asyncio
import aiohttp
import json
import websockets

async def send_rest(api, method, params):
    ts = int(time.time() * 1000)
    signature_payload = f'{ts}{method.upper()}{api}'.encode()
    
    if params:
        signature_payload += json.dumps(params).encode()
        
    signature = hmac.new(secret.encode(), signature_payload, 'sha256').hexdigest()
    async with aiohttp.ClientSession(headers={'FTX-KEY': key, 'FTX-SIGN': signature, 'FTX-TS': str(ts)}) as session:
        if params:
            async with session.request(method, rest + api, json=params) as response:
                return await response.json()
        else:
            async with session.request(method, rest + api) as response:
                return await response.json()


### Creating subaccount to account orders easier

In [217]:
api = '/api/subaccounts/'
method = 'GET'
params = {}

await send_rest(api, method, params)

{'success': True,
 'result': [{'nickname': 'arbitrage',
   'special': False,
   'deletable': True,
   'editable': True,
   'competition': False},
  {'nickname': 'carry',
   'special': False,
   'deletable': True,
   'editable': True,
   'competition': False}]}

In [218]:
api = '/api/subaccounts/'
method = 'POST'
params = {
  "nickname": 'arbitrage'
}

await send_rest(api, method, params)

{'success': False, 'error': 'Subaccount with that name already exists'}

### Transfer 100$

In [ ]:
api = '/api/subaccounts/transfer'
method = 'POST'
params = {
  "coin": "USD",
  "size": 100,
  "source": 'main',
  "destination": "arbitrage",
}

await send_rest(api, method, params)

### Hisotrical orders

Должны выбрать две определенные пары валют. К примеру LTC-PERP и LTC

In [251]:
api = '/api/orders/history?market=MEDIA/USD'
method = 'GET'
params = {}

data = await send_rest(api, method, params)
data

{'success': True,
 'result': [{'id': 46794261659,
   'clientId': '777',
   'market': 'MEDIA/USD',
   'type': 'limit',
   'side': 'buy',
   'price': 141.0,
   'size': 0.5,
   'status': 'closed',
   'filledSize': 0.0,
   'remainingSize': 0.0,
   'reduceOnly': False,
   'liquidation': False,
   'avgFillPrice': None,
   'postOnly': False,
   'ioc': False,
   'createdAt': '2021-05-10T10:31:13.899125+00:00',
   'future': None},
  {'id': 46792506566,
   'clientId': None,
   'market': 'MEDIA/USD',
   'type': 'limit',
   'side': 'buy',
   'price': 142.0,
   'size': 0.5,
   'status': 'closed',
   'filledSize': 0.0,
   'remainingSize': 0.0,
   'reduceOnly': False,
   'liquidation': False,
   'avgFillPrice': None,
   'postOnly': False,
   'ioc': False,
   'createdAt': '2021-05-10T10:19:49.945767+00:00',
   'future': None},
  {'id': 46791995272,
   'clientId': None,
   'market': 'MEDIA/USD',
   'type': 'limit',
   'side': 'buy',
   'price': 142.0,
   'size': 0.5,
   'status': 'closed',
   'filledSi

### Placing orders

In [250]:
async def buy(market, price, size, client_id, reduce_only=False, type='limit'):
    api = '/api/orders'
    method = 'POST'
    params = {"market": market, "side": "buy","price": price,"type": type,"size": size,"clientId": client_id, "reduceOnly": reduce_only}
    return await send_rest(api, method, params)


async def sell(market, price, size, client_id, reduce_only, type='limit'):
    api = '/api/orders'
    method = 'POST'
    params = {"market": market, "side": "sell","price": price,"type": type,"size": size,"clientId": client_id, "reduceOnly": reduce_only}
    return await send_rest(api, method, params)


async def cancel_order_by_client_id(client_id):
    api = f'/api/orders/by_client_id/{client_id}'
    method = 'DELETE'
    params = {}
    return await send_rest(api, method, params)


async def get_order_status_by_client_id(client_id, params={}):
    api = f'/api/orders/by_client_id/{client_id}'

    method = 'GET'
    return await send_rest(api, method, params)

In [ ]:
'MEDIA/USD'

In [293]:
async def first_leg(symbol):
    async with websockets.connect('wss://ftx.com/ws/') as client:

        await client.send(
            json.dumps(
                {'op': 'login', 'args': {'key': key, 'sign': hmac.new(
                    secret.encode(), f'{ts}websocket_login'.encode(), 'sha256').hexdigest(), 'time': ts}}
            ).encode()
        )

        await client.send(json.dumps({'op': 'subscribe', 'channel': 'ticker', 'market': symbol}))
        
        print(await client.recv())
        
        try:
            while True:
                data = json.loads(await client.recv())
                print(data)
        except:
            pass
        
        
async def wrapper():
    await asyncio.gather(
        asyncio.create_task(
            first_leg('BTC-PERP')),
        asyncio.create_task(
            first_leg('BTC/USD'))
    )
        
# await first_leg('MEDIA-PERP')
# await first_leg('MEDIA/USD')
await wrapper()

{"type": "error", "code": 400, "msg": "Invalid login credentials"}
{"type": "error", "code": 400, "msg": "Invalid login credentials"}
{'type': 'subscribed', 'channel': 'ticker', 'market': 'BTC/USD'}
{'type': 'subscribed', 'channel': 'ticker', 'market': 'BTC-PERP'}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58210.0, 'ask': 58211.0, 'bidSize': 0.119, 'askSize': 0.7119, 'last': 58210.0, 'time': 1620647605.2719676}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58210.0, 'ask': 58211.0, 'bidSize': 0.119, 'askSize': 1.2056, 'last': 58210.0, 'time': 1620647605.29501}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58167.0, 'ask': 58168.0, 'bidSize': 0.1155, 'askSize': 0.1378, 'last': 58168.0, 'time': 1620647605.283934}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58167.0, 'ask': 58168.0, 'bidSize': 0.1155, 'askSize': 0.1378, 'last': 58168.0, 'time': 1620647605.3053

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58205.0, 'ask': 58206.0, 'bidSize': 0.01, 'askSize': 1.3764, 'last': 58205.0, 'time': 1620647605.8909407}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58205.0, 'ask': 58206.0, 'bidSize': 0.01, 'askSize': 2.4757, 'last': 58205.0, 'time': 1620647605.9142282}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58159.0, 'ask': 58168.0, 'bidSize': 0.0801, 'askSize': 1.6207, 'last': 58167.0, 'time': 1620647605.9078531}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58205.0, 'ask': 58206.0, 'bidSize': 0.01, 'askSize': 2.4757, 'last': 58205.0, 'time': 1620647605.942526}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58205.0, 'ask': 58206.0, 'bidSize': 0.01, 'askSize': 2.4757, 'last': 58205.0, 'time': 1620647605.964194}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid'

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.2822, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647606.9908733}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.2822, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.0401316}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58159.0, 'ask': 58160.0, 'bidSize': 0.0846, 'askSize': 0.006, 'last': 58160.0, 'time': 1620647607.0466614}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.2822, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.0662544}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.2822, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.0915413}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data'

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.4369, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.8679287}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58159.0, 'ask': 58160.0, 'bidSize': 0.0846, 'askSize': 0.006, 'last': 58160.0, 'time': 1620647607.8718657}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.4369, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.8929017}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58159.0, 'ask': 58165.0, 'bidSize': 0.0846, 'askSize': 0.3, 'last': 58160.0, 'time': 1620647607.9005852}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5229, 'askSize': 0.8161, 'last': 58201.0, 'time': 1620647607.9158692}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5929, 'askSize': 0.7525, 'last': 58201.0, 'time': 1620647608.730001}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58164.0, 'ask': 58165.0, 'bidSize': 0.0786, 'askSize': 0.0519, 'last': 58165.0, 'time': 1620647608.7442248}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.6429, 'askSize': 0.7525, 'last': 58200.0, 'time': 1620647608.7808714}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58164.0, 'ask': 58165.0, 'bidSize': 0.0681, 'askSize': 0.0519, 'last': 58165.0, 'time': 1620647608.7944498}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58164.0, 'ask': 58165.0, 'bidSize': 0.0681, 'askSize': 0.3639, 'last': 58165.0, 'time': 1620647608.8215842}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data':

...

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5632, 'askSize': 0.3983, 'last': 58201.0, 'time': 1620647613.8541791}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5632, 'askSize': 0.3983, 'last': 58201.0, 'time': 1620647613.8797305}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5632, 'askSize': 0.6308, 'last': 58201.0, 'time': 1620647613.908798}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58160.0, 'ask': 58161.0, 'bidSize': 0.0632, 'askSize': 0.1099, 'last': 58161.0, 'time': 1620647613.9100804}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5632, 'askSize': 0.6308, 'last': 58201.0, 'time': 1620647613.9301414}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data'

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5688, 'askSize': 3.0256, 'last': 58201.0, 'time': 1620647614.704473}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5688, 'askSize': 3.0256, 'last': 58201.0, 'time': 1620647614.7290592}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58152.0, 'ask': 58159.0, 'bidSize': 0.0741, 'askSize': 0.032, 'last': 58153.0, 'time': 1620647614.7328184}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5688, 'askSize': 3.0256, 'last': 58201.0, 'time': 1620647614.7573106}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58158.0, 'ask': 58159.0, 'bidSize': 0.0908, 'askSize': 0.032, 'last': 58153.0, 'time': 1620647614.7549834}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {

{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5914, 'askSize': 1.601, 'last': 58201.0, 'time': 1620647615.5457091}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58158.0, 'ask': 58159.0, 'bidSize': 0.1138, 'askSize': 0.032, 'last': 58153.0, 'time': 1620647615.5403323}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5941, 'askSize': 1.601, 'last': 58201.0, 'time': 1620647615.5706959}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5941, 'askSize': 1.601, 'last': 58201.0, 'time': 1620647615.6008263}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5941, 'askSize': 1.601, 'last': 58201.0, 'time': 1620647615.620742}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'

{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58158.0, 'ask': 58159.0, 'bidSize': 0.017, 'askSize': 0.0908, 'last': 58166.0, 'time': 1620647616.3746665}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58158.0, 'ask': 58159.0, 'bidSize': 0.017, 'askSize': 0.0908, 'last': 58166.0, 'time': 1620647616.3996801}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5864, 'askSize': 1.6364, 'last': 58200.0, 'time': 1620647616.4018435}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {'bid': 58200.0, 'ask': 58201.0, 'bidSize': 0.5864, 'askSize': 1.6364, 'last': 58200.0, 'time': 1620647616.4237537}}
{'channel': 'ticker', 'market': 'BTC/USD', 'type': 'update', 'data': {'bid': 58158.0, 'ask': 58159.0, 'bidSize': 0.017, 'askSize': 1.4679, 'last': 58166.0, 'time': 1620647616.4231534}}
{'channel': 'ticker', 'market': 'BTC-PERP', 'type': 'update', 'data': {

CancelledError: 

In [290]:
import json
import hmac

ts = int(time.time() * 1000)
async with websockets.connect('wss://ftx.com/ws/') as client:
    first_leg = 'MEDIA-PERP'
    second_leg = 'MEDIA/USD'

    await client.send(
        json.dumps(
            {'op': 'login', 'args': {'key': key, 'sign': hmac.new(
                secret.encode(), f'{ts}websocket_login'.encode(), 'sha256').hexdigest(), 'time': ts}}
        ).encode()
    )
    
    await client.send(json.dumps({'op': 'subscribe', 'channel': 'ticker', 'market': first_leg}))
    await client.send(json.dumps({'op': 'subscribe', 'channel': 'ticker', 'market': second_leg}))
    print(await client.recv())
    try:
        while True:
            data = json.loads(await client.recv())
            print(data)
    except:
        pass


    

{"type": "subscribed", "channel": "ticker", "market": "MEDIA-PERP"}
{'type': 'subscribed', 'channel': 'ticker', 'market': 'MEDIA/USD'}
{'channel': 'ticker', 'market': 'MEDIA-PERP', 'type': 'update', 'data': {'bid': 142.0, 'ask': 142.2, 'bidSize': 36.44, 'askSize': 11.23, 'last': 142.0, 'time': 1620647353.4527323}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647353.5152187}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647353.592664}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647353.6169064}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 't

{'channel': 'ticker', 'market': 'MEDIA-PERP', 'type': 'update', 'data': {'bid': 142.0, 'ask': 142.2, 'bidSize': 36.44, 'askSize': 11.23, 'last': 142.0, 'time': 1620647363.1543083}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647363.835179}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647363.9279003}}
{'channel': 'ticker', 'market': 'MEDIA-PERP', 'type': 'update', 'data': {'bid': 142.0, 'ask': 142.2, 'bidSize': 36.44, 'askSize': 11.23, 'last': 142.0, 'time': 1620647364.1946507}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidSize': 1.43, 'askSize': 20.8, 'last': 141.9, 'time': 1620647364.2348557}}
{'channel': 'ticker', 'market': 'MEDIA/USD', 'type': 'update', 'data': {'bid': 141.6, 'ask': 141.9, 'bidS

In [285]:
data

{'channel': 'orderbook',
 'market': 'MEDIA-PERP',
 'type': 'update',
 'data': {'time': 1620646047.1907732,
  'checksum': 1924079140,
  'bids': [],
  'asks': [],
  'action': 'update'}}

In [261]:
time.time() * 1000

1620645039171.7441